In [4]:
   
"""
代码来自
 https://github.com/beaverden/pdftoc/tree/main
"""

def get_tree_pages(root, info,count, depth=0):
    """
        Recursively iterate the outline tree
        Find the pages pointed by the outline item
        and get the assigned physical order id

        Decrement with padding if necessary
    """
        
    if isinstance(root,dict):
        # print(root)
        page = root['/Page'].getObject()
        print(id(page))
        t = root['/Title']
        title = t
        if isinstance(t, PyPDF2.generic.ByteStringObject):
            title = t.original_bytes.decode('utf8')
        title = title.strip()
        title = title.replace('\n', '')
        title = title.replace('\r', '')
        

        page_num = info['all_pages'].get(id(page), 0)
        if page_num == 0:
            print('Not found page number for /Page!', page)
        elif page_num < info['padding']:
            page_num = 0
        else:
            page_num -= info['padding']

        
        str_val = '%-5d' % page_num
        str_val += '\t' * depth
        str_val += title + '\t' + '%3d' % page_num
        print("str_val: ",str_val)
        
        with open(outline, "a+") as file:
            file.write(str_val+'\n')
            
            file.close() 
        
        return 
    for elem in root:
        get_tree_pages(elem, info, count,depth+1)

def recursive_numbering(obj, info):
    """
        Recursively iterate through all the pages in order and assign them a physical
        order number
    """
    # print(id(obj), obj)
    if obj['/Type'] == '/Page':
        obj_id = id(obj)
        if obj_id not in info['all_pages']:
            info['all_pages'][obj_id] = info['current_page_id']
        info['current_page_id'] += 1
        return
    elif obj['/Type'] == '/Pages':
        for page in obj['/Kids']:
            recursive_numbering(page.getObject(), info)
        
    
def create_text_outline(pdf_path, page_number_padding):
    # print('Running the script for [%s] with padding [%d]' % (pdf_path, page_number_padding))
    # creating an object 
    
    count=0
    
    with open(pdf_path, 'rb') as file:
        fileReader = PyPDF2.PdfFileReader(file)

        info = {
            'all_pages': {},
            'current_page_id': 1, 
            'padding': page_number_padding
        }
        # out=fileReader.getOutlines()
        # for item in out:
            # print(item)
            # chapter_title.append(item)
        pages = fileReader.trailer['/Root']['/Pages'].getObject()
        recursive_numbering(pages, info)
        # for page_num, page in enumerate(pages['/Kids']):
        #    page_obj = page.getObject()
        #    all_pages[id(page_obj)] = page_num + 1 # who starts counting from 0 anyways?
        get_tree_pages(fileReader.getOutlines(), info,count)
        
    return 

In [6]:
import PyPDF2
import numpy as np

def extract_chap_title():
    outline="outline.txt"

    with open('outline.txt','w') as f:
        f.write('\n')

    # create_text_outline("example.pdf", 0)


In [15]:


"""
根据章节/子章节名字的字符串提取文件内容，内容提取的范围精确到页（不是很精确）
首先需要根据pdf2c.py提取章节名字和他们的开始页并把结果写入.txt file 
process txt file to get respective page number for retrieving information
"""

def locate_chapter_by_chapter_name(file):
    """
    from outline.txt file build dictionary contains all extracted chapter/subchapter
    names and their starting page number 
    return the dict
    """
    dict={}
    dict['page']=[]
    dict['title']=[]
    with open(file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line != '\n':
                info=line.split(' ')
                # print(info)
                str=''
                for i in range(len(info)-1):
                    if i is not 0:
                        str+=info[i]
                # print(str) 
            
                # print(len(info))
                dict['title'].append(str.replace('\t',''))
                dict['page'].append(info[0])
                
    return dict

def find_start_end_page(dict,chap_name): #chapter and sub-chapter have equal importance
    """
    use string of chapter name return the starting page number and ending page number of the chapter
    return the start page, end page and the page number in between in a list 
    """
    start=0
    end=0
    for i in range(len(dict['title'])):
        if str(dict['title'][i]) == str(chap_name):
            print(dict['title'][i])
            start=dict['page'][i]
            end=dict['page'][i+1]
    # print(start,end)
    if start==end==0:
        print('no valide (sub)chapter can be found')
        return 0
    return [i for i in range(int(start),int(end)+1)]#[inclusive,inclusive]

def find_text_by_page_nums(pdf_file, page_nums):

    fileobj=open(pdf_file,'rb')
    pdfReader = PyPDF2.PdfFileReader(fileobj)
    for i in page_nums:
        pageObj = pdfReader.getPage(i-1)
        text = pageObj.extractText().split("  ")
        for i in range(len(text)):
            print(text[i],end="\n\n")

<>:27: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:27: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/ipykernel_54103/733490619.py:27: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if i is not 0:


In [16]:
dict=locate_chapter_by_chapter_name('outline.txt')
print(dict)

{'page': ['2', '2', '4', '6', '7', '7', '7', '7', '7', '7', '8', '8', '8', '8', '9', '9', '9', '10', '10', '10', '11', '11', '11', '12', '12', '13', '13', '13', '14', '14', '15', '16', '16', '16', '17', '18', '19', '19', '20', '20', '20', '20', '20', '22', '22', '23', '24', '25', '26', '27'], 'title': ['山东能源集团有限公司', '2023年跟踪评级报告', '主要财务数据：', '声明', '山东能源集团有限公司', '2023年跟踪评级报告', '一、跟踪评级原因', '二、主体概况', '三、存续债券情况', '四、宏观经济和政策环境', '五、行业分析', '六、基础素质分析', '1．产权状况', '2．企业规模', '3．过往债务履约情况', '七、管理分析', '八、重大事项', '九、经营分析', '1.经营概况', '2.煤炭业务', '3.贸易业务', '4.其他业务', '（1）化工业务', '（2）金融业务', '5.经营效率', '6.在建工程及未来发展', '十、财务分析', '1．财务概况', '2．资产质量', '（1）流动资产', '（2）非流动资产', '3．资本结构', '（1）所有者权益', '（2）负债', '4．盈利能力', '5．现金流', '6．偿债指标', '7．公司本部财务分析', '十一、外部支持', '1.支持能力', '2.支持可能性', '十二、存续债券偿还能力分析', '十三、结论', '附件1-1截至2022年底公司股权结构图', '附件1-2截至2023年3月底公司组织结构图', '附件1-3截至2022年底公司主要子公司情况', '附件2-1主要财务数据及指标（合并口径）', '附件2-2主要财务数据及指标（公司本部）', '附件3主要财务指标的计算公式', '附件4-1主体长期信用等级设置及含义']}


In [18]:
print('the pdf contains chapters:', dict['title'])

the pdf contains chapters: ['山东能源集团有限公司', '2023年跟踪评级报告', '主要财务数据：', '声明', '山东能源集团有限公司', '2023年跟踪评级报告', '一、跟踪评级原因', '二、主体概况', '三、存续债券情况', '四、宏观经济和政策环境', '五、行业分析', '六、基础素质分析', '1．产权状况', '2．企业规模', '3．过往债务履约情况', '七、管理分析', '八、重大事项', '九、经营分析', '1.经营概况', '2.煤炭业务', '3.贸易业务', '4.其他业务', '（1）化工业务', '（2）金融业务', '5.经营效率', '6.在建工程及未来发展', '十、财务分析', '1．财务概况', '2．资产质量', '（1）流动资产', '（2）非流动资产', '3．资本结构', '（1）所有者权益', '（2）负债', '4．盈利能力', '5．现金流', '6．偿债指标', '7．公司本部财务分析', '十一、外部支持', '1.支持能力', '2.支持可能性', '十二、存续债券偿还能力分析', '十三、结论', '附件1-1截至2022年底公司股权结构图', '附件1-2截至2023年3月底公司组织结构图', '附件1-3截至2022年底公司主要子公司情况', '附件2-1主要财务数据及指标（合并口径）', '附件2-2主要财务数据及指标（公司本部）', '附件3主要财务指标的计算公式', '附件4-1主体长期信用等级设置及含义']


In [20]:
name="5.经营效率"
page_nums =find_start_end_page(dict,str(name))
page_nums

5.经营效率


[12, 13]

In [21]:
print(find_text_by_page_nums("example.pdf",page_nums))

跟踪评级报告
www.lhratings.com 12生产能力。
产量方面，2022年，除醋酸产量同比小幅
下降，公司甲醇、尿素和焦炭产量同比均有所
提升。其中，受焦炭产能提升影响，公司焦炭
产量同比增长22.03%，产能利用率有所下降。
2022年，除焦炭外，公司其他主要化工产品产
能利用率均超过100%。
销量方面，2022年，因公司部分醋酸产品
用于内部其他化工产品的生产，产销率较低；
其他主要化工产品产销率均可达到95%以上。
销售价格方面，2022年，受市场环境变化
影响，公司醋酸产品销售价格同比下降38.93%；
但受原材料成本增加等因素影响，甲醇、尿素
和焦炭的销售价格分别同比上升3.58%、5.63%
和12.75%。
表6煤化工主要产品产销情况
（单位：万吨/年、万吨、%、元/吨）
产品 项目名称 2020年 2021年 2022年
甲醇产能 535 535 531
产量 507.09516.53531.29
产能利用率 94.7896.55100.05
销量 359.34400.72508.81
产销率 70.8677.5895.77
含税销售价 1346.271992.852064.13
醋酸产能 100 100 100
产量 107.23109.16103.58
产能利用率 107.23109.16103.58
销量 75.9275.7271.16
产销率 70.8069.3768.70
含税销售价 2233.665645.663447.68
尿素产能 52 52 52
产量 54.5059.8462.79
产能利用率 104.81115.08120.75
销量 54.5059.8963.27
产销率 100.00100.08100.76
含税销售价 1281.221970.332081.20
焦炭产能 495 435 563
产量 452.42383.51468.01
产能利用率 91.4088.6183.13
销量 370.99395.10493.17
产销率 82.00103.02105.38
含税销售价 1926.822360.872661.80
注：产能利用率超过100%的原因：设计产能按300天统计，在实际生产组织过程中，企业优化组织生产，实际生产实际达到330天，产能利用率
超过100%
资料来源：公司提供
公司煤化工板块客户较